In [1]:
import numpy as np
from scipy.stats import norm

# function

In [2]:
ITERATION_MAX_ERROR = 0.00001

def priceEuropeanOption(option_type_flag, S, X, T, r, b, v):
    '''
    Black-Scholes
    '''

    d1 = (np.log(S / X) + (b + v**2 / 2) * T) / (v * (T)**0.5)
    d2 = d1 - v * (T)**0.5

    if option_type_flag == 'Call':
        bsp = S * np.exp((b - r) * T) * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    else:
        bsp = X * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp((b - r) * T) * norm.cdf(-d1)
        
    return bsp

def priceAmericanOption(option_type_flag, S, X, T, r, b, v):
    '''
    Barone-Adesi-Whaley
    '''
    
    if option_type_flag == 'Call':
        return approximateAmericanCall(S, X, T, r, b, v)
    elif option_type_flag == 'Put':
        return approximateAmericanPut(S, X, T, r, b, v)
    
def approximateAmericanCall(S, X, T, r, b, v):
    '''
    Barone-Adesi And Whaley
    '''

    if b >= r:
        return priceEuropeanOption('Call', S, X, T, r, b, v)
    else:
        Sk = Kc(X, T, r, b, v)
        N = 2 * b / v**2                                           
        k = 2 * r / (v**2 * (1 - np.exp(-1 * r * T)))
        d1 = (np.log(Sk / X) + (b + (v**2) / 2) * T) / (v * (T**0.5))
        Q2 = (-1 * (N - 1) + ((N - 1)**2 + 4 * k))**0.5 / 2
        a2 = (Sk / Q2) * (1 - np.exp((b - r) * T) * norm.cdf(d1))
        if S < Sk:
            return priceEuropeanOption('Call', S, X, T, r, b, v) + a2 * (S / Sk)**Q2
        else:
            return S - X
def approximateAmericanPut(S, X, T, r, b, v):
    '''
    Barone-Adesi-Whaley
    '''

    Sk = Kp(X, T, r, b, v)
    N = 2 * b / v**2
    k = 2 * r / (v**2 * (1 - np.exp(-1 * r * T)))
    d1 = (np.log(Sk / X) + (b + (v**2) / 2) * T) / (v * (T)**0.5)
    Q1 = (-1 * (N - 1) - (((N - 1)**2 + 4 * k))**0.5) / 2
    a1 = -1 * (Sk / Q1) * (1 - np.exp((b - r) * T) * norm.cdf(-1 * d1))

    if S > Sk:
        return priceEuropeanOption('Put', S, X, T, r, b, v) + a1 * (S / Sk)**Q1
    else:
        return X - S
    
def Kc(X, T, r, b, v):

    N = 2 * b / v**2
    m = 2 * r / v**2
    q2u = (-1 * (N - 1) + ((N - 1)**2 + 4 * m)**0.5) / 2
    su = X / (1 - 1 / q2u)
    h2 = -1 * (b * T + 2 * v * (T)**0.5) * X / (su - X)
    Si = X + (su - X) * (1 - np.exp(h2))

    k = 2 * r / (v**2 * (1 - np.exp(-1 * r * T)))
    d1 = (np.log(Si / X) + (b + v**2 / 2) * T) / (v * (T)**0.5)
    Q2 = (-1 * (N - 1) + ((N - 1)**2 + 4 * k)**0.5) / 2
    LHS = Si - X
    RHS = priceEuropeanOption('Call', Si, X, T, r, b, v) + (1 - np.exp((b - r) * T) * norm.cdf(d1)) * Si / Q2
    bi = np.exp((b - r) * T) * norm.cdf(d1) * (1 - 1 / Q2) + (1 - np.exp((b - r) * T) * norm.pdf(d1) / (v * (T)**0.5)) / Q2

    E = ITERATION_MAX_ERROR
    
    while np.abs(LHS - RHS) / X > E:
        Si = (X + RHS - bi * Si) / (1 - bi)
        d1 = (np.log(Si / X) + (b + v**2 / 2) * T) / (v * (T)**0.5)
        LHS = Si - X
        RHS = priceEuropeanOption('Call', Si, X, T, r, b, v) + (1 - np.exp((b - r) * T) * norm.cdf(d1)) * Si / Q2
        bi = np.exp((b - r) * T) * norm.cdf(d1) * (1 - 1 / Q2) + (1 - np.exp((b - r) * T) * norm.cdf(d1) / (v * (T)**0.5)) / Q2
    
    return Si
def Kp(X, T, r, b, v):

    N = 2 * b / v**2
    m = 2 * r / v**2
    q1u = (-1 * (N - 1) - ((N - 1)**2 + 4 * m)**0.5) / 2
    su = X / (1 - 1 / q1u)
    h1 = (b * T - 2 * v * (T)**0.5) * X / (X - su)
    Si = su + (X - su) * np.exp(h1)

    k = 2 * r / (v**2 * (1 - np.exp(-1 * r * T)))
    d1 = (np.log(Si / X) + (b + v**2 / 2) * T) / (v * (T)**0.5)
    Q1 = (-1 * (N - 1) - ((N - 1)**2 + 4 * k)**0.5) / 2
    LHS = X - Si
    RHS = priceEuropeanOption( 'Put', Si, X, T, r, b, v) - (1 - np.exp((b - r) * T) * norm.cdf(-1 * d1)) * Si / Q1
    bi = -1 * np.exp((b - r) * T) * norm.cdf(-1 * d1) * (1 - 1 / Q1) - (1 + np.exp((b - r) * T) * norm.pdf(-d1) / (v * (T)**0.5)) / Q1
    
    E = ITERATION_MAX_ERROR
    
    while np.abs(LHS - RHS) / X > E:
        Si = (X - RHS + bi * Si) / (1 + bi)
        d1 = (np.log(Si / X) + (b + v**2 / 2) * T) / (v * (T)**0.5)
        LHS = X - Si
        RHS = priceEuropeanOption('Put', Si, X, T, r, b, v) - (1 - np.exp((b - r) * T) * norm.cdf(-1 * d1)) * Si / Q1
        bi = -np.exp((b - r) * T) * norm.cdf(-1 * d1) * (1 - 1 / Q1) - (1 + np.exp((b - r) * T) * norm.cdf(-1 * d1) / (v * (T)**0.5)) / Q1
        
    return Si

# parameters

In [3]:
S = 40
X_call = 40  # 選擇權履約價格
X_put = 40  # 選擇權履約價格
t = 1  # 選擇權到期時間（年）
r = 0.06  # 無風險利率
b = 0.02  # 常數（持有成本 b=r-q）
sigma = 0.2  # 波動率


In [4]:
price_c = priceAmericanOption('Call', S, X_call, t, r, b, sigma)
price_p = priceAmericanOption('Put', S, X_put, t, r, b, sigma)
print(f'call price : {price_c}, put_price : {price_p}')

call price : 3.4339696034703824, put_price : 2.781123533637977
